##### 1. Установить implicit

In [1]:
#!pip install implicit

##### 2. Скачиваем lastfm_small.tsv

In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender

In [4]:
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, precision_score
from math import sqrt

In [5]:
col_names = ['user', 'artist-mbid', 'artist-name', 'total-plays']
data = pd.read_csv('lastfm_small.tsv',
                   sep='\t',
                   encoding='utf-8',
                   header=None,
                   names=col_names)

In [6]:
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [7]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data["plays"] = data["total-plays"].astype(np.double)
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user","total-plays"], axis=1, inplace=True)
data.head()

,user_id,artist_id,plays
0,1,15531,2137.0
1,1,63469,1099.0
2,1,46858,897.0
3,1,15968,717.0
4,1,48969,706.0


In [8]:
data.describe()

,user_id,artist_id,plays
count,1000000.000000,1000000.000000,1000000.000000
mean,10232.925996,33678.492236,216.606950
std,5912.022447,19230.330182,604.378024
min,1.000000,1.000000,1.000000
25%,5118.000000,17298.000000,34.000000
50%,10237.000000,34544.000000,94.000000
75%,15347.000000,49488.000000,225.000000
max,20465.000000,66799.000000,135392.000000


##### 3. Разбить датасет на обучающую и тестовую выборки

In [9]:
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [10]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 26
всего пользователей: 20465


In [11]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 0
нет в тестовой выборке, но есть в обучающей: 0
всего пользователей: 20465


In [13]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [14]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

In [15]:
plays_KNN = coo_matrix((
    train_data["plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays_KNN.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799741
Доля ненулевых элементов: 0.000584978533112608
Среднее значение ненулевых элементов: 216.87803649902344
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


In [16]:
plays_ALS = coo_matrix((
    train_data["plays"].astype('double'),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays_ALS.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799741
Доля ненулевых элементов: 0.000584978533112608
Среднее значение ненулевых элементов: 216.8780717757374
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


##### 4. Построить на обучающей выборке модель kNN

In [17]:
model_KNN = CosineRecommender()

In [18]:
model_KNN.fit(plays_KNN)

##### 5. Построить на обучающей выборке модель ALS

In [19]:
model_ALS = AlternatingLeastSquares()

In [20]:
model_ALS.fit(plays_ALS)

##### 6. Получить рекомендации на тестовой выборке для kNN

In [21]:
user_plays_KNN = plays_KNN.T.tocsr()

In [22]:
print("получаем рекомендации для всех пользователей kNN")
predict_kNN = []
start = time.time()
user_plays = plays_KNN.T.tocsr()
test_unique = test_data['user_id'].unique()
for user_id, username in enumerate(test_unique):
    for artist_id, score in model_KNN.recommend(user_id, user_plays):
        predict_kNN.append((user_id,artist_id, score))  
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))  

получаем рекомендации для всех пользователей kNN
получили рекомендации для всех пользователей за 27.550312280654907 секнуд


In [23]:
df_predict_KNN = pd.DataFrame(predict_kNN, columns=['user_id','artist_id','plays'])

In [24]:
merge_KNN = pd.merge(test_data, df_predict_KNN, on=['user_id', 'artist_id'])

In [25]:
mean_predict_KNN = merge_KNN["plays_y"].mean()
mean_predict_KNN

715.462934904304

In [26]:
mean_truth_KNN = merge_KNN["plays_x"].mean()
mean_truth_KNN 

313.3049441786284

In [27]:
bin_predict_KNN = merge_KNN["plays_y"].apply(lambda x: 1 if x > mean_predict_KNN else 0 )
bin_truth_KNN = merge_KNN["plays_x"].apply(lambda x: 1 if x > mean_truth_KNN else 0 )

##### 7. Получить рекомендации на тестовой выборке для ALS

In [28]:
user_plays_ALS = plays_ALS.T.tocsr()

In [29]:
print("получаем рекомендации для всех пользователей ALS")
predict_ALS = []
start = time.time()
user_plays = plays_ALS.T.tocsr()
test_unique = test_data['user_id'].unique()
for user_id, username in enumerate(test_unique):
    for artist_id, score in model_ALS.recommend(user_id, user_plays):
        predict_ALS.append((user_id,artist_id, score))  
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start)) 

получаем рекомендации для всех пользователей ALS
получили рекомендации для всех пользователей за 144.4758813381195 секнуд


In [30]:
df_predict_ALS = pd.DataFrame(predict_ALS, columns=['user_id','artist_id','plays'])

In [31]:
merge_ALS = pd.merge(test_data, df_predict_ALS, on=['user_id', 'artist_id'])

In [32]:
mean_predict_ALS = merge_ALS["plays_y"].mean()
mean_predict_ALS

0.9922654861913619

In [33]:
mean_truth_ALS = merge_ALS["plays_x"].mean()
mean_truth_ALS 

291.3263891934564

In [43]:
bin_predict_ALS = merge_ALS["plays_y"].apply(lambda x: 1 if x > mean_predict_ALS else 0 )
bin_truth_ALS = merge_ALS["plays_x"].apply(lambda x: 1 if x > mean_truth_ALS else 0 )

##### 8. Сравнить метрики качества обученных моделей на тестовой выборке

In [44]:
print( 'RMSE_KNN: ' + str(sqrt(mean_squared_error(merge_KNN['plays_x'],merge_KNN['plays_y']))))

RMSE_KNN: 1140.092442215645


In [45]:
print( 'RMSE_AlS: ' + str(sqrt(mean_squared_error(merge_ALS['plays_x'],merge_ALS['plays_y']))))

RMSE_AlS: 1073.4407825963838


In [46]:
print( 'Average precision KNN: ' + str(average_precision_score(bin_predict_KNN, bin_truth_KNN)))

Average precision KNN: 0.500712764102


In [47]:
print( 'Average precision ALS: ' + str(average_precision_score(bin_predict_ALS, bin_truth_ALS)))

Average precision ALS: 0.683376459138


In [48]:
print( 'Precision KNN: ' + str(precision_score(bin_predict_KNN, bin_truth_KNN)))

Precision KNN: 0.653705953827


In [49]:
print( 'Precision ALS: ' + str(precision_score(bin_predict_ALS, bin_truth_ALS)))

Precision ALS: 0.865888689408


In [50]:
print( 'ROC AUC KNN: ' + str(roc_auc_score(bin_predict_KNN, bin_truth_KNN)))

ROC AUC KNN: 0.713407688294


In [51]:
print( 'ROC AUC ALS: ' + str(roc_auc_score(bin_predict_ALS, bin_truth_ALS)))

ROC AUC ALS: 0.644145287741
